Recommended to view this example notebook on [`nbviewer` here](https://nbviewer.org/github/SMTG-UCL/ShakeNBreak/blob/main/tutorials/ShakeNBreak_Example_Workflow.ipynb) for cleaner (scrollable) outputs

# `ShakeNBreak` applied to Cd vacancies in CdTe ($V_{Cd}$)

In this notebook we follow the full `ShakeNBreak` (`SnB`) workflow, where we:
- Apply the defect distortions
- Parse the geometry relaxation results
- Re-generate any energy-lowering distortions found for _some_ (but not all) charge states for a given defect
- Plot the final energies to demonstrate what energy-lowering defect distortions have been identified
- Then continue our defect calculations, confident we have obtained the ground-state structures. 

## Table of contents
* [Generate defects with doped/pymatgen](#generate)
* [Apply SnB to defects](#SnB)
* [Send to HPCs and run calculations](#HPCs)
* [Plot energies of final structures](#plot)
* [(Optional) Analyse defect distortions](#analyse)
* [(Optional) Generate input files for other codes](#other)

In [ ]:
import sys
import os
import doped
import pymatgen
import ase
import shakenbreak
import numpy as np
from importlib.metadata import version

# check versions
print("Doped version:" , version('doped') )
print("Pymatgen version:" , version('pymatgen') )
print("Ase version:" , version('ase') )
print("ShakeNBreak version:" , version('shakenbreak') )

<a id='generate'></a>

## 1. Generate defects with `doped`/`pymatgen`

In [ ]:
from doped.pycdt.core.defectsmaker import ChargedDefectsStructures
from pymatgen.core.structure import Structure

# Specify perfect (bulk) supercell structure
bulk_supercell = Structure.from_file("../tests/data/vasp/CdTe/CdTe_Bulk_Supercell_POSCAR")  

# Let's generate the vacancies
def_structs = ChargedDefectsStructures(
    bulk_supercell,
    cellmax=bulk_supercell.num_sites,
    antisites_flag=False,
    include_interstitials=False,
)

In [ ]:
defect_dict = def_structs.defects
# only using these species for example purpose:
V_Cd_dict = {"vacancies": [defect_dict["vacancies"][0]]}
V_Cd_dict["vacancies"][0]["charges"] = [0,]

#### Rationale for `SnB`   

Defect distortions often follow the change in electron count when introducing that defect to the system. For the neutral Cd vacancy ($V_{Cd}^0$) for example, the removal of Cd and its two valence electrons means that local distortions are likely to involve two neighbouring Te atoms moving closer/further apart to accommodate the broken bonds. For the singly-charged vacancy, we are likely to have just one neighbouring Te moving, etc. This isn't always the case, but typically points us in the right direction to search the PES, and has been confirmed to yield the best performance (see SI of _Identifying the ground state structures of point defects in solids_ Mosquera-Lois, Kavanagh, Walsh and Scanlon 2022).

So, the `SnB` method involves distorting the initial bond lengths around the defect for a mesh of trial distortions, with the number of neighbours to distort dictated by the change in valence electron count, performing coarse $\Gamma$-only (`vasp_gam`) relaxations and then comparing the final energies, to see if we identify any lower energy defect structures.

<a id='SnB'></a>

## 2. Apply the `SnB` method to your defects

The default settings and parameter choices in this package have been tested and have performed best thus far (i.e. wider distortion ranges leading to the ground-state structure with lowest computational cost) – see SI of _Identifying the ground state structures of point defects in solids_ Mosquera-Lois, Kavanagh, Walsh and Scanlon 2022.

If you encounter improved performance with non-default parameter choices, we'd love to know! Please get in touch via GitHub or by email: sean.kavanagh.19@ucl.ac.uk & i.mosquera-lois22@imperial.ac.uk

If you are investigating defects in hard/ionic/close-packed materials, or systems involving spectator ions (like A in ABX$_3$), there are some extra considerations for boosting the performance & efficiency of `SnB` listed on the [Miscellaneous Tips & Tricks](https://shakenbreak.readthedocs.io/en/latest/Tips.html) docs page.

### 2.1 Generating distorted structures

In [ ]:
from shakenbreak import input, energy_lowering_distortions
from shakenbreak.input import Distortions

In [ ]:
# In order to determine the number of the defect nearest neighbours to distort (based on the change 
# in valence electrons mentioned above), SnB uses the oxidation states of atoms in our material:
# If not specified, the code will guess these, otherwise you can specify as such:
# oxidation_states = {"Cd": +2, "Te": -2}  # specify atom oxidation states

# Create an instance of Distortion class with the defect dictionary and the distortion parameters
# If distortion parameters are not specified, the default values are used
Dist = Distortions(
    defects_dict=dict(V_Cd_dict),
    #oxidation_states=oxidation_states,  # explicitly specify oxidation states
)

In [ ]:
# We can check the distortion parameters using some of the class properties
print("Bond distortions:", Dist.bond_distortions)
print("Rattle standard deviation:", Dist.stdev, "A")

In [ ]:
# You can restrict the ions that are distorted to a certain element using the keyword distorted_elements
# We can check it using the class attribute
print("User defined elements to distort:", Dist.distorted_elements)
# If None, it means no restrictions so nearest neighbours are distorted (recommended default, 
# unless you have reason to suspect otherwise; see shakenbreak.readthedocs.io/en/latest/Tips.html)

To see the optional parameters that can be tuned in the distortion functions, look at the docstrings:  
_(If you are viewing this in `nbviewer`, long output cells like this and printed dictionaries/structures below are scrollable!)_

In [ ]:
input.Distortions?

If we're only interested in generating distorted structures, but not in writting `VASP`/other codes input files, we can use the class method `Distortions.apply_distortions()` to do this.

In [ ]:
defects_dict, distortion_metadata = Dist.apply_distortions()

In [ ]:
defects_dict["vac_1_Cd"].keys()

In [ ]:
# The output dictionary contains information about each defect:
print("Keys for each defect entry:", defects_dict["vac_1_Cd"].keys())

# As well as the distorted structures for each charge state of all defects
# We can access the distorted structures of vac_1_Cd_0 like this:
print("\nUndistorted and distorted structures:")
defects_dict["vac_1_Cd"]["charges"][0]["structures"]

### 2.2 Generating `VASP` input files for the distorted structures

If we want to generate `VASP` input files, we can use the class method `Distortions.write_vasp_files()` (instead of `Distortions.apply_distortions()`)

In [ ]:
Dist.write_vasp_files()

Using the `incar_settings` optional argument for `Distortions.write_vasp_files()` above, we can also specify some custom `INCAR` tags to match our converged `ENCUT` for this system and optimal `NCORE` for the HPC we will run the calculations on. More information on the distortions generated can be obtained by setting `verbose = True`.

Our distorted structures and VASP input files have now been generated in the `vac_1_Cd_X` folders.

For the recommended default coarse structure-searching `INCAR` settings, either have a look at the `incar.yaml` file in the `input_files` folder or at the generated files:

In [ ]:
!cat ./vac_1_Cd_0/Bond_Distortion_-10.0%/INCAR

Note that the `NELECT` `INCAR` tag (number of electrons) is automatically determined based on the choice of `POTCAR`s. The default in `ShakeNBreak` is to use the [`MPRelaxSet` `POTCAR` choices](https://github.com/materialsproject/pymatgen/blob/master/pymatgen/io/vasp/MPRelaxSet.yaml), but if you're using different ones, make sure to set `potcar_settings` in `apply_shakenbreak()`, so that NELECT is then set accordingly. This requires the `pymatgen` config file `$HOME/.pmgrc.yaml` to be properly set up as detailed on the [GitHub `README`](https://github.com/SMTG-UCL/ShakeNBreak) and [Installation](https://shakenbreak.readthedocs.io/en/latest/Installation.html) docs page.

For generating the input files for other electronic structure codes (`Quantum Espresso`, `FHI-aims`, `CP2K`, `CASTEP`), see the [(Optional) Generate input files for other codes](#other) section at the end of this notebook.

<a id='HPCs'></a>

## 3. Send to HPCs and run relaxations

#### a) For `VASP` users:

Then parse the energies obtained by running the `snb-parse` command from the top-level folder containing your defect folders (e.g. `vac_1_Cd_0` etc. (with subfolders: `vac_1_Cd_0/Bond_Distortion_10.0%` etc.)). This will parse the energies and store them in a `vac_1_Cd_0.yaml` etc file in the defect folders, to allow easy plotting and analysis.

It is also recommended to parse the final structures (`CONTCAR`s files if using `VASP`) obtained with each distortion relaxation for further structural analysis, which is done automatically when downloaded to your local folders as below. 

Copying these data to your local PC can be done quickly from your local folder top-level folder (containing `vac_1_Cd_0` etc) with the following code:

```bash
for defect in ./*{_,_-}[0-9]/; do cd $defect; scp {remote_machine}:{path to ShakeNBreak folders}/${defect}${defect%?}.yaml .; for distortion in (Bond_Distortion|Unperturbed|Rattled)*/; do scp {remote_machine}:{path to ShakeNBreak folders}/${defect}${distortion}CONTCAR ${distortion}; done; cd ..; done
```
making sure to change `{remote_machine}` and `{path to ShakeNBreak folders}` to the correct values in your case.

#### b) If using `CP2K`, `Quantum Espresso`, `CASTEP` or `FHI-aims`:
Then parse the energies obtained by running the `snb-parse` command from the top-level folder containing your defect folders (e.g. `vac_1_Cd_0` etc. (with subfolders: `vac_1_Cd_0/Bond_Distortion_10.0%` etc.)) and setting the `--code` option (e.g. `snb-parse --code cp2k`). This will parse the energies and store them in a `vac_1_Cd_0.yaml` etc file in the defect folders, to allow easy plotting and analysis. 

It is also recommended to parse the final structures obtained with each relaxation for further structural analysis. Depending on the code the structure information is read from:
* `CP2K`: restart file
* `Quantum Espresso`: output file
* `CASTEP`: output file (i.e. `.castep`)
* `FHI-aims`: output file

**_For demonstration purposes_** in this example notebook, we'll use some (fake) example data:  
(Don't do this if you're actually running the calculations and have downloaded the data as instructed above)

In [ ]:
!cp -r ../tests/data/example_results/vac_1_Cd* .
!cp ../tests/data/vasp/CdTe/distortion_metadata.json .
# may need to change path if you've moved the example notebook elsewhere

<a id='plot'></a>

## 4. Plot energies vs distortion
To see if `SnB` found any energy-lowering distortions, we can plot the results using the functions in `shakenbreak.plotting`.

In [ ]:
from shakenbreak import energy_lowering_distortions, plotting

In [ ]:
defect_charges_dict = energy_lowering_distortions.read_defects_directories()
low_energy_defects = energy_lowering_distortions.get_energy_lowering_distortions(defect_charges_dict)

These functions give us some info about whether any energy-lowering defect distortions were identified, and we can see the results clearer by plotting:

In [ ]:
figs = plotting.plot_all_defects(defect_charges_dict)

This prints the distortion plots for all defects where a significant energy lowering distortion, relative to the standard unperturbed relaxation, was identified. The threshold energy difference to consider as 'significant' is controlled by the `min_e_diff` optional parameter (default = 0.05 eV).

### Can also add a colorbar 
These plots can be made more informative by adding a colorbar showing the structural similarity between the relaxed structures.   
For this you need the `CONTCAR`'s obtained with each distortion (as mentioned above).

For the colorbar structure comparison metric, you can either use:
* summed root mean squared displacement (`metric` = `disp`)
* maximum distance between matching sites (`metric` = `max_dist`, default).   

In [ ]:
figs = plotting.plot_all_defects(
    defect_charges_dict,
    add_colorbar=True
)

In [ ]:
figs = plotting.plot_all_defects(
    defect_charges_dict,
    add_colorbar=True,
    metric="disp"
)

So for these example results, we find energy lowering distortions for $V_{Cd}^0$ (at -0.3, -0.4 and -0.6 bond distortion factors) and $V_{Cd}^{-1}$ (from 0.2 to 0.6 bond distortion factors). We should re-test these distorted structures for the $V_{Cd}$ charge states where these distortions were not found, in case they also give lower energies. 

Of course, this is not necessary if these structures were already found in the distortion tests for the other charge states, and so the `get_energy_lowering_distortions()` function automatically performs structure comparisons to determine which distortions should be tested in other charge states of the same defect, and which have already been found (see docstring for more details). 

In the output of `get_energy_lowering_distortions()` (which we saved to `low_energy_defects` in the earlier cell), we get a dictionary of defects for which bond distortion found an energy-lowering distortion (which is missed with normal unperturbed relaxation), of the form {defect: [list of distortion dictionaries (with corresponding charge states, energy lowering, distortion factors, structures and charge states for which these structures weren't found)]}.

For example, our results with $V_{Cd}$ show that we found an energy-lowering distortion for the neutral case (`subdict["charges"]`) which wasn't found with the -2 or -1 charge states (`subdict["excluded_charges"]`) – and so we'll test this distorted structures with those charge states – and also an energy-lowering distortion for -1 which wasn't found with 0 or -2 charge states.

In [ ]:
for index, subdict in enumerate(low_energy_defects["vac_1_Cd"]):
    print(f"Energy lowering distortion number {index}")
    print("Found for charge states:", subdict["charges"])  # Charge state for which the energy lowering was found
    print(f"Not found in:", subdict["excluded_charges"], "\n")

This generates the new distorted structures and VASP inputs, to do our quick second round of structure testing (energy-lowering distortions found for at least one, but not all charge states for a given defect):

In [ ]:
energy_lowering_distortions.write_distorted_inputs(low_energy_defects)

Note here the nomenclature we use for the distorted structures we've imported from other charge states (i.e. `Bond_Distortion_-60.0%_from_0` refers to the structure obtained from relaxing the -60% distortion of the neutral (q = 0) charge state).

Again we run the calculations on the HPCs, then parse and download the data to our local folders (but again in this example notebook we'll use our fake example data **_for demonstration purposes_** as in the next cell, but don't do this if you're actually running the calculations!)

In [ ]:
!cp ./vac_1_Cd_0/vac_1_Cd_0_additional_distortions.yaml ./vac_1_Cd_0/vac_1_Cd_0.yaml
!cp ./vac_1_Cd_-1/vac_1_Cd_-1_additional_distortions.yaml ./vac_1_Cd_-1/vac_1_Cd_-1.yaml
!cp ./vac_1_Cd_-2/vac_1_Cd_-2_additional_distortions.yaml ./vac_1_Cd_-2/vac_1_Cd_-2.yaml
!cp ./vac_1_Cd_0/Bond_Distortion_-60.0%/CONTCAR ./vac_1_Cd_-1/Bond_Distortion_-60.0%_from_0/
!cp ./vac_1_Cd_0/Bond_Distortion_-60.0%/CONTCAR ./vac_1_Cd_-2/Bond_Distortion_-60.0%_from_0/
!cp ./vac_1_Cd_-1/Unperturbed/CONTCAR ./vac_1_Cd_-2/Bond_Distortion_20.0%_from_-1/
!cp ./vac_1_Cd_-1/Unperturbed/CONTCAR ./vac_1_Cd_0/Bond_Distortion_20.0%_from_-1/

Then re-parse with the same `get_energy_lowering_distortions()` function from before:

In [ ]:
low_energy_defects = energy_lowering_distortions.get_energy_lowering_distortions(defect_charges_dict)

Finally we can replot the results from all our distortion tests:

In [ ]:
figs = plotting.plot_all_defects(defect_charges_dict)

In this example case, for $V_{Cd}^{0}$ the distorted structure originally found for the -1 charge state comes out lower energy than the $V_{Cd}^{0}$ unperturbed relaxation, but still higher energy than the previously identified ground-state at -0.3, -0.4 and -0.6 distortion factors. 

For $V_{Cd}^{-1}$, the distorted structure originally found for the neutral (0) charge state comes out lower energy than the previously identified ground-state at distortion factors >0.2.

We now continue our defect calculations using the ground-state `CONTCAR`s we've obtained for each defect, with our fully-converged `INCAR` and `KPOINTS` settings, to get our final defect formation energies (confident that we've identified the ground-state defect structure!). The `energy_lowering_distortions.write_groundstate_structure()` function automatically writes these lowest-energy structures to our defect folders:

In [ ]:
energy_lowering_distortions.write_groundstate_structure()

In [ ]:
!head vac_1_Cd_0/groundstate_POSCAR  # groundstate structure from -60% distortion relaxation

In [ ]:
!diff vac_1_Cd_0/groundstate_POSCAR vac_1_Cd_0/Bond_Distortion_-60.0%/CONTCAR  # groundstate structure from -60% distortion relaxation

<a id='analyse'></a>

## 5. *Optional*: Analyse the defect distortions found with `SnB`

If we want to analyse in more detail the defect distortions identified with our structure searching, we can use some of the functions from `shakenbreak.analysis`:

In [ ]:
from shakenbreak import analysis

# Parse all structures obtained with distortions and unperturbed relaxation.   
# This gives a dictionary matching initial distortion to final structure
vac_1_Cd_0 = analysis.get_structures("vac_1_Cd_0")

In [ ]:
# Can then analyse a chosen final structure with:
df = analysis.analyse_structure("vac_1_Cd_0", vac_1_Cd_0["Unperturbed"])
df = analysis.analyse_structure("vac_1_Cd_0", vac_1_Cd_0[-0.4])

We can also compare the structural similarity between all structures with `compare_structures()`. It prints the summed root mean squared displacement,
maximum distance between paired sites, and energy (relative to unperturbed structure) of all final structures:

In [ ]:
defect_energies = analysis.get_energies("vac_1_Cd_0")
structure_comparison = analysis.compare_structures(
    vac_1_Cd_0,
    defect_energies
)

Highly favourable distortions are often driven by some kind of rebonding. For most vacancies and interstitials, this entails formation of new homoionic bonds between the defect neighbours.
We can quickly check for these reconstructions using `analysis.get_homoionic_bonds()`

In [ ]:
bonds = analysis.get_homoionic_bonds(
    structure=vac_1_Cd_0[-0.4], # Structure to analyse
    element="Te", # we're looking for Te-Te bonds
    radius=2.8, # maximum bond distance between 2 Te
    verbose=False, # don't print bond distances
)
print(bonds)
print("So two of the vacancy neighbours formed a Te-Te bond to compensate for the charge deficiency")

For defects that can result in polarons, we can check the sites with significant magnetization using `analysis.get_site_magnetizations()`

In [ ]:
df = analysis.get_site_magnetizations(
    defect_species="vac_1_Ti_0", # neutral Ti vacancy in anatase TiO2
    distortions=["Unperturbed", -0.4],
    defect_site=[0.0, 0.16666666666666669, 0.25],
    threshold=0.3, # to filter sites with significant magnetization
    orbital_projections=False, # don't show orbital projections
)

display(df["Unperturbed"]) # 4 holes localised on 4 of the vacancy neighbours
print("So we have 4 holes localised on 4 of the oxygen ions neighbouring the vacancy")

In [ ]:
!cp -r ../tests/data/vasp/vac_1_Ti_0 .

In [ ]:
df = analysis.get_site_magnetizations(
    defect_species="vac_1_Ti_0", # neutral Ti vacancy in anatase TiO2
    distortions=["Unperturbed", -0.4],
    defect_site=[0.0, 0.16666666666666669, 0.25],
    threshold=0.3, # to filter sites with significant magnetization
    orbital_projections=False, # don't show orbital projections
)

display(df["Unperturbed"]) # 4 holes localised on 4 of the vacancy neighbours
print("So we have 4 holes localised on 4 of the oxygen ions neighbouring the vacancy")

As printed below, no significant magnetization is found for -40.0% distortion. This configuration was found to be significantly more stable than the polaronic solution, so we can quickly use `analysis.get_homoionic_bonds` to see why:

In [ ]:
bonds = analysis.get_homoionic_bonds(
    structure = Structure.from_file("./vac_1_Ti_0/Bond_Distortion_-40.0%/CONTCAR"),
    element="O",
    radius=2.0,
    verbose=False,
)
print(bonds)
print("So the formation of an O-O bond drived this distortion")

In [ ]:
bonds = analysis.get_homoionic_bonds(
    structure = Structure.from_file("./vac_1_Ti_0/Bond_Distortion_-40.0%/CONTCAR"),
    element="O",
    radius=2.0,
    verbose=False,
)
print(bonds)
print("So the formation of an O-O bond drived this distortion")

See the [documentation](https://shakenbreak.readthedocs.io/en/latest/) for more info and optional parameter choices etc

<a id='other'></a>

## 6. *Optional*: Input File Generation for Other Codes

#### a) `Quantum Espresso`

In [ ]:
# check the arguments of the `write_espresso_files` method
Dist.write_espresso_files?

In [ ]:
# oxidation_states = {"Cd": +2, "Te": -2}  # explicitly specify atom oxidation states

# Create an instance of Distortion class with the defect dictionary and the distortion parameters
# If distortion parameters are not specified, the default values are used
Dist = Distortions(
    defects_dict=dict(V_Cd_dict),
    #oxidation_states=oxidation_states, # explicitly specify atom oxidation states
    bond_distortions=[-0.3, 0.3] # For demonstration purposes, just doing 2 distortions
)

pseudopotentials = { # Your chosen pseudopotentials
    'Cd': 'Cd_pbe_v1.uspp.F.UPF',
    'Te': 'Te.pbe-n-rrkjus_psl.1.0.0.UPF'}

defects_dict, distortion_metadata = Dist.write_espresso_files(
    pseudopotentials=pseudopotentials,
)

And for the default coarse structure-searching input settings, either have a look at the `qe_input.yaml` file in the `input_files` folder or at the generated files:

In [ ]:
!cat ./vac_1_Cd_0/Bond_Distortion_30.0%/espresso.pwi

#### b) `CP2K`

In [ ]:
Dist.write_cp2k_files?

In [ ]:
oxidation_states = {"Cd": +2, "Te": -2}  # explicitly specify atom oxidation states

# Create an instance of Distortion class with the defect dictionary and the distortion parameters
# If distortion parameters are not specified, the default values are used
Dist = Distortions( 
            defects_dict=dict(V_Cd_dict), 
            oxidation_states=oxidation_states, # explicitly specify atom oxidation states
            bond_distortions=[-0.3, 0.3] # For demonstration purposes, just doing 2 distortions
            )

defects_dict, distortion_metadata = Dist.write_cp2k_files()

And for the default coarse structure-searching input settings, either have a look at the `cp2k_input.yaml` file in the `input_files` folder or at the generated files:

In [ ]:
!cat ./vac_1_Cd_0/Bond_Distortion_30.0%/cp2k_input.inp

#### c) `CASTEP`

In [ ]:
Dist.write_castep_files?

In [ ]:
oxidation_states = {"Cd": +2, "Te": -2}  # explicitly specify atom oxidation states

# Create an instance of Distortion class with the defect dictionary and the distortion parameters
# If distortion parameters are not specified, the default values are used
Dist = Distortions(
    defects_dict=dict(V_Cd_dict),
    oxidation_states=oxidation_states,  # explicitly specify atom oxidation states
    bond_distortions=[0.3] # For demonstration purposes, just doing 2 distortions
)
# If we don't specify the input_file, only the structure files (in .cell format) are written
defects_dict, distortion_metadata = Dist.write_castep_files()

And for the default coarse structure-searching input settings, either have a look at the `castep.param` file in the `input_files` folder or at the generated files:

In [ ]:
!cat ./vac_1_Cd_0/Bond_Distortion_30.0%/castep.param

#### d) `FHI-aims`

In [ ]:
Dist.write_fhi_aims_files?

In [ ]:
oxidation_states = {"Cd": +2, "Te": -2}  # specify atom oxidation states

# Create an instance of Distortion class with the defect dictionary and the distortion parameters
# If distortion parameters are not specified, the default values are used
Dist = Distortions( 
    defects_dict=dict(V_Cd_dict), 
    oxidation_states=oxidation_states,
    bond_distortions=[0.3] # For demonstration purposes, just doing 2 distortions
)
# If we don't specify the input_file, only the structure files (in .cell format) are written
defects_dict, distortion_metadata = Dist.write_fhi_aims_files()

And for the default coarse structure-searching input settings have a look at the generated files:

In [ ]:
!cat ./vac_1_Cd_0/Bond_Distortion_30.0%/control.in

These can be modified with the `ase_calculator` option of the `Distortion.write_fhi_aims_files()` method

### (Developer) `nbviewer` Settings:

In [ ]:
%%html
<style>
.nbviewer div.output_area {
  max-height: 1000px; /* or value of your choosing; https://github.com/jupyter/nbviewer/issues/750 */
}
</style>

### (Developer) `nbviewer` Settings:

In [9]:
%%html
<style>
.nbviewer div.output_area {
  max-height: 1000px; /* or value of your choosing; https://github.com/jupyter/nbviewer/issues/750 */
}
</style>